<a href="https://colab.research.google.com/github/TeamPrigge/MyProject/blob/master/Order_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [ ]:

# You have to use the encoding as latin1 to read this file as there are some special character in this file...
df = pd.read_csv (r'Order_list_lage.csv',encoding='latin1')
df.head()
# with open('order_list_Bestellung.csv', newline='') as f:
#     reader = csv.reader(f)
#     for column in reader:
#         objects=(column[1])
#         print(objects)




sorted_by_product = df.sort_values(['Product'], ascending=False)

b= sorted_by_product['Product'].values #.astype(str)
c= b.astype(str)
a = sorted_by_product['Product'].value_counts()
# print(b[1])
# print(c)
type(c)
print(c)
# len(c)





sorted_by_product["Product"].head(10)

plt.figure(figsize=(20,40))
a.plot(kind='barh')

sorted_by_product["Product"].tail(10)

#This package provides helpers for computing similarities between arbitrary sequences. 
#Included metrics are Levenshtein, Hamming, Jaccard, and Sorensen distance, 
#plus some bonuses. All distance computations are implemented in pure Python, 
#and most of them are also implemented in C.
pip install Distance

import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Import your data to a Pandas.DataFrame
df = pd.read_csv('Order_list_lage.csv',encoding='latin1')

# Grab the column you'd like to group, filter out duplicate values
# and make sure the values are Unicode
# type(b)
vals = df['Product'].astype('U')#unique().astype('U')
# print(vals)


# Write a function for cleaning strings and returning an array of ngrams
def ngrams_analyzer(string):
    string = re.sub(r'[,-./]', r'', string)
    ngrams = zip(*[string[i:] for i in range(4)])  # N-Gram length is 5
    return [''.join(ngram) for ngram in ngrams]

# Construct your vectorizer for building the TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

# Build the matrix!!!
tfidf_matrix = vectorizer.fit_transform(vals)

pip install sparse_dot_topn

# Import IGN's awesome_cossim_topn module
from sparse_dot_topn import awesome_cossim_topn

# The arguments for awesome_cossim_topn are as follows:
### 1. Our TF-IDF matrix
### 2. Our TF-IDF matrix transposed (allowing us to build a pairwise cosine matrix)
### 3. A top_n filter, which allows us to filter the number of matches returned, which isn't useful for our purposes
### 4. This is our similarity threshold. Only values over 0.8 will be returned
cosine_matrix = awesome_cossim_topn(
  tfidf_matrix,
  tfidf_matrix.transpose(),
  vals.size,
  0.35
)


# Build a coordinate matrix from a cosine matrix
coo_matrix = cosine_matrix.tocoo()

# Instaniate our lookup hash table
group_lookup = {}


def find_group(row, col):
    # If either the row or the col string have already been given
    # a group, return that group. Otherwise return none
    if row in group_lookup:
        return group_lookup[row]
    elif col in group_lookup:
        return group_lookup[col]
    else:
        return None


def add_vals_to_lookup(group, row, col):
    # Once we know the group name, set it as the value
    # for both strings in the group_lookup
    group_lookup[row] = group
    group_lookup[col] = group


def add_pair_to_lookup(row, col):
    # in this function we'll add both the row and the col to the lookup
    group = find_group(row, col)  # first, see if one has already been added
    if group is not None:
        # if we already know the group, make sure both row and col are in lookup
        add_vals_to_lookup(group, row, col)
    else:
        # if we get here, we need to add a new group.
        # The name is arbitrary, so just make it the row
        add_vals_to_lookup(row, row, col)

# for each row and column in coo_matrix
# if they're not the same string add them to the group lookup
for row, col in zip(coo_matrix.row, coo_matrix.col):
    if row != col:
        # Note that what is passed to add_pair_to_lookup is the string at each index
        # (eg: the names in the legal_name column) not the indices themselves
        add_pair_to_lookup(vals[row], vals[col])

df['Group'] = df['Product'].map(group_lookup).fillna(df['Product'])

df.to_csv('./dol-data-grouped.csv')

# You have to use the encoding as latin1 to read this file as there are some special character in this file...
df = pd.read_csv ('./dol-data-grouped.csv',encoding='latin1')
df.head()

sorted_by_product = df.sort_values(['Group'], ascending=False)

# b= sorted_by_product['Group'].values #.astype(str)
# c= b.astype(str)
a = sorted_by_product['Group'].value_counts()
sorted_by_product["Group"].head(10)

plt.figure(figsize=(10,20))
a.plot(kind='barh')


